In [1]:
# import necessary libraries
import pandas as pd
import sys
import os
from sodapy import Socrata  #pull data with Tyler api

client = Socrata('data.cityofchicago.org',
                 os.environ['CHICAGO_DATA_APP_TOKEN'],  #secrets stay secret, sorry not sorry
                 username="bk2988a@american.edu",
                 password=os.environ['CHICAGO_DATA_PWD'])  #pwds are not for sharing

Keys established.


In [ ]:
results = client.get("ijzp-q8t2", limit=2000)

In [ ]:
df.head()